# Single horizon

In [51]:
import pandas as pd
import ast

# load prediction for Moirai for 2023-2024
df_moirai = pd.read_csv("analysis/future_data/temporary_2023_2024/es_future_final_moirai_2023_2024.csv")

# drop the first context_lenght rows because they have missing values for Result (nan)
# we keep the last 3 rows even if they don't have predictions. They will serve as a reference for the last 3 predicted values.
df_moirai = df_moirai.drop(df_moirai.index[:383])

# fix the horizon for predictions
horizon = 2
horizon_index = horizon - 1

prediction_length = 3

last_index = df_moirai.index[-1]
last_index_for_es = last_index - prediction_length

spread = 1

list_of_gains = []
list_of_losses = []

for index_in_df, row in df_moirai.iterrows():
    current_price = float(row["Close"])
    if index_in_df > last_index_for_es:
        print("We estimate the gains for all values. We will not use the last 3 values because we don't have the real values for all horizons to evalue them.")
        break
    predictions_list = ast.literal_eval(row["Result"])
    future_prices = df_moirai.loc[index_in_df:].iloc[1:prediction_length+1]["Close"].tolist()
    
    predicted_price = predictions_list[horizon_index]
    future_price = future_prices[horizon_index]
    
    if predicted_price > current_price: # we predict a UP
        presumed_gain = predicted_price - current_price
        difference = 0
        if future_price > current_price:
            difference = future_price - current_price
            difference = difference - spread
            list_of_gains.append((index_in_df, difference))
        elif future_price < current_price:
            difference = future_price - current_price
            difference = difference - spread
            list_of_losses.append((index_in_df, difference))
        else:
            print("No gain or loss. Except for the fees.")

    elif predicted_price < current_price: # we predict a DOWN
        presumed_gain = current_price - predicted_price
        difference = 0
        if future_price < current_price:
            difference = current_price - future_price
            difference = difference - spread
            list_of_gains.append((index_in_df, difference))
        elif future_price > current_price:
            difference = current_price - future_price
            difference = difference - spread
            list_of_losses.append((index_in_df, difference))
        else:
            print("No gain or loss. Except for the fees.")
        
    else: # we predict a STAY
        print("We should have a stay position.")

print("Processing finished.")


total_gain = sum([gain for index, gain in list_of_gains])
total_loss = sum([loss for index, loss in list_of_losses])
result = total_gain + total_loss
total_predictions = len(list_of_gains) + len(list_of_losses)
mean_result = result / total_predictions

print("Model: ", "moirai")
print("Horizon (in hours): ", horizon)
print("Spread (in points): ", spread)
print(f"Total gain (in points): {total_gain}")
print(f"Total loss (in points): {total_loss}")

print("Number of gains: ", len(list_of_gains))
print("Number of losses: ", len(list_of_losses))

print(f"Result: {result} points")

if total_gain > abs(total_loss):
    print(f"Mean result (positive) {mean_result} points")
else:
    print(f"Mean result (negative) {mean_result} points")

# Generate Excel File
data = {
    "Model": ["moirai"],
    "Horizon (in hours)": [horizon],
    "Spread (in points)": [spread],
    "Total gain (in points)": [total_gain],
    "Total loss (in points)": [total_loss],
    "Number of gains": [len(list_of_gains)],
    "Number of losses": [len(list_of_losses)],
    "Result (in points)": [result],
    "Mean result (per prediction, in points)": [mean_result]
}

df = pd.DataFrame(data)

# Save to Excel
file_path = "model_performance.xlsx"
df.to_excel(file_path, index=False, engine='openpyxl')
print(f"Data saved to {file_path}")


No gain or loss. Except for the fees.
No gain or loss. Except for the fees.
No gain or loss. Except for the fees.
No gain or loss. Except for the fees.
No gain or loss. Except for the fees.
No gain or loss. Except for the fees.
No gain or loss. Except for the fees.
No gain or loss. Except for the fees.
No gain or loss. Except for the fees.
No gain or loss. Except for the fees.
No gain or loss. Except for the fees.
No gain or loss. Except for the fees.
No gain or loss. Except for the fees.
No gain or loss. Except for the fees.
No gain or loss. Except for the fees.
No gain or loss. Except for the fees.
No gain or loss. Except for the fees.
No gain or loss. Except for the fees.
No gain or loss. Except for the fees.
No gain or loss. Except for the fees.
No gain or loss. Except for the fees.
No gain or loss. Except for the fees.
No gain or loss. Except for the fees.
No gain or loss. Except for the fees.
No gain or loss. Except for the fees.
No gain or loss. Except for the fees.
No gain or l

# Multiple horizons

In [54]:
import pandas as pd
import ast

# Load the CSV (same file for all horizons)
df_moirai = pd.read_csv("analysis/future_data/temporary_2023_2024/es_future_final_moirai_2023_2024.csv")

# Drop the first context_length rows because they have missing values for Result (NaN).
# Keep the last 3 rows even if they don't have predictions; they will serve as a reference
# for the last 3 predicted values.
df_moirai = df_moirai.drop(df_moirai.index[:383])

prediction_length = 3

# We'll use a single list to accumulate the results for each horizon.
results = []

# We want to evaluate horizons 1 through 3.
for horizon in [1, 2, 3]:
    horizon_index = horizon - 1

    last_index = df_moirai.index[-1]
    last_index_for_es = last_index - prediction_length

    spread = 1

    list_of_gains = []
    list_of_losses = []

    for index_in_df, row in df_moirai.iterrows():
        current_price = float(row["Close"])
        if index_in_df > last_index_for_es:
            print("We estimate the gains for all values. We will not use the last 3 values because we don't have the real values for all horizons to evaluate them.")
            break

        # Convert the string representation of the list in the "Result" column to a Python list
        predictions_list = ast.literal_eval(row["Result"])
        future_prices = df_moirai.loc[index_in_df:].iloc[1:prediction_length+1]["Close"].tolist()

        predicted_price = predictions_list[horizon_index]
        future_price = future_prices[horizon_index]

        # If predicted price is greater than the current price, we predict an UP movement
        if predicted_price > current_price:
            if future_price > current_price:
                difference = future_price - current_price
                difference -= spread
                list_of_gains.append((index_in_df, difference))
            elif future_price < current_price:
                difference = future_price - current_price
                difference -= spread
                list_of_losses.append((index_in_df, difference))
            else:
                print("No gain or loss. Except for the fees.")

        # If predicted price is less than the current price, we predict a DOWN movement
        elif predicted_price < current_price:
            if future_price < current_price:
                difference = current_price - future_price
                difference -= spread
                list_of_gains.append((index_in_df, difference))
            elif future_price > current_price:
                difference = current_price - future_price
                difference -= spread
                list_of_losses.append((index_in_df, difference))
            else:
                print("No gain or loss. Except for the fees.")

        else:
            print("We should have a stay position.")

    print("Processing finished for horizon:", horizon)

    # Summarize the results for this horizon
    total_gain = sum([gain for index, gain in list_of_gains])
    total_loss = sum([loss for index, loss in list_of_losses])
    result = total_gain + total_loss
    total_predictions = len(list_of_gains) + len(list_of_losses)
    mean_result = result / total_predictions

    print("Model:", "moirai")
    print("Horizon (in hours):", horizon)
    print("Spread (in points):", spread)
    print(f"Total gain (in points): {total_gain}")
    print(f"Total loss (in points): {total_loss}")

    print("Number of gains:", len(list_of_gains))
    print("Number of losses:", len(list_of_losses))

    print(f"Result: {result} points")

    if total_gain > abs(total_loss):
        print(f"Mean result (positive) {mean_result} points")
    else:
        print(f"Mean result (negative) {mean_result} points")

    # Store these horizon results
    results.append({
        "Model": "moirai",
        "Horizon (in hours)": horizon,
        "Spread (in points)": spread,
        "Leverage": 8,
        "The position equals to 1 contract": True,
        "Total gain (in points)": total_gain,
        "Total loss (in points)": total_loss,
        "Number of gains": len(list_of_gains),
        "Number of losses": len(list_of_losses),
        "Result (in points)": result,
        "Mean result (per prediction, in points)": mean_result,
        "Total predictions": len(list_of_gains) + len(list_of_losses),
    })

# Create a DataFrame from the accumulated results
df_results = pd.DataFrame(results)

# Save all horizon data in a single Excel file
file_path = "model_performance_horizons.xlsx"
df_results.to_excel(file_path, index=False, engine='openpyxl')
print(f"Data saved to {file_path}")

No gain or loss. Except for the fees.
No gain or loss. Except for the fees.
No gain or loss. Except for the fees.
No gain or loss. Except for the fees.
No gain or loss. Except for the fees.
No gain or loss. Except for the fees.
No gain or loss. Except for the fees.
No gain or loss. Except for the fees.
No gain or loss. Except for the fees.
No gain or loss. Except for the fees.
No gain or loss. Except for the fees.
No gain or loss. Except for the fees.
No gain or loss. Except for the fees.
No gain or loss. Except for the fees.
No gain or loss. Except for the fees.
No gain or loss. Except for the fees.
No gain or loss. Except for the fees.
No gain or loss. Except for the fees.
No gain or loss. Except for the fees.
No gain or loss. Except for the fees.
No gain or loss. Except for the fees.
No gain or loss. Except for the fees.
No gain or loss. Except for the fees.
No gain or loss. Except for the fees.
No gain or loss. Except for the fees.
No gain or loss. Except for the fees.
No gain or l